In [23]:
import numpy as np
import matplotlib.pyplot as plt
import dmol
from dmol import color_cycle

In [24]:
# make our colors (nothing to do with the model)

# so color cycle just gives random hex values for colors
for c in color_cycle:
    print(c)

vertex_colors = []
for c in color_cycle:
    # turns the hex color from base 8 to base 16 to get RGB 
    hex_color = int(c[1:], 16)
    r = hex_color // 256**2
    hex_color = hex_color - r * 256**2
    g = hex_color // 256
    hex_color = hex_color - g * 256
    b = hex_color
    # and then normalizes to a scale tof 0-1
    vertex_colors.append((r / 256, g / 256, b / 256))
vertex_colors = np.array(vertex_colors)

def z6_fxn(x):
    return vertex_colors[x]

z6_fxn(0)

print(vertex_colors.shape)

#444444
#1BBC9B
#a895bb
#F06060
#F3B562
#80cedb
(6, 3)


In [25]:
# make weights be 3x3 matrices at each group element
# 3x3 so that we have 3 color channels in and 3 out

weights = np.random.normal(size=(6, 3, 3))

# returns the weight for the channel specified
def z6_omega(x):
    return weights[x]

z6_omega(3)
print(weights)

[[[-0.01702041  0.37915174  2.25930895]
  [-0.04225715 -0.955945   -0.34598178]
  [-0.46359597  0.48148147 -1.54079701]]

 [[ 0.06326199  0.15650654  0.23218104]
  [-0.59731607 -0.23792173 -1.42406091]
  [-0.49331988 -0.54286148  0.41605005]]

 [[-1.15618243  0.7811981   1.49448454]
  [-2.06998503  0.42625873  0.67690804]
  [-0.63743703 -0.39727181 -0.13288058]]

 [[-0.29779088 -0.30901297 -1.67600381]
  [ 1.15233156  1.07961859 -0.81336426]
  [-1.46642433  0.52106488 -0.57578797]]

 [[ 0.14195316 -0.31932842  0.69153875]
  [ 0.69474914 -0.72559738 -1.38336396]
  [-1.5829384   0.61037938 -1.18885926]]

 [[-0.50681635 -0.59631404 -0.0525673 ]
  [-1.93627981  0.1887786   0.52389102]
  [ 0.08842209 -0.31088617  0.09740017]]]


In [26]:
# returns the inverse of g, if g is an element of the group 
def z6_inv(g):
    return (6 - g) % 6

# defining the group operation, which is just addition and then modulo 6 for the cycle
def z6_prod(g1, g2):
    return (g1 + g2) % 6

def conv(f, p):
    def out(u):
        g = np.arange(6)
        # einsum is so we can do matrix product for elements of f and g,
        # since we have one matrix per color
        # so we ignore the first dimension of the second matrix, which is p(g) or z6_omega, the (6, 3, 3) matrix that has basically 6 channels of 3 x 3 kernels. 
        c = np.sum(np.einsum("ij,ijk->ik", f(z6_prod(u, z6_inv(g))), p(g)), axis=0)
        return c

    return out

conv(z6_fxn, z6_omega)(0)

array([-6.29256306,  0.29995792, -1.60243073])

In [27]:
def z6_fxn_trans(g, f):
    return lambda h: f(z6_prod(z6_inv(g), h))

z6_fxn(0), z6_fxn_trans(2, z6_fxn)(0)

(array([0.265625, 0.265625, 0.265625]),
 array([0.94921875, 0.70703125, 0.3828125 ]))

In [28]:
# transform and then convolution
trans_element = 2
# resulting function after translation
trans_input_fxn = z6_fxn_trans(trans_element, z6_fxn)
# passing the resulting function into the convolution with z6_omega, which are random filters
trans_input_out = conv(trans_input_fxn, z6_omega) 

In [29]:
# convolution and then ransform
output_fxn = conv(z6_fxn, z6_omega)
trans_output_out = z6_fxn_trans(trans_element, output_fxn)

print("g -> psi[f(g)], g -> psi[Tgf(g)], g-> Tg psi[f(g)]")
for i in range(6):
    print(
        i,
        np.round(conv(z6_fxn, z6_omega)(i), 2),
        np.round(trans_input_out(i), 2),
        np.round(trans_output_out(i), 2),
    )

g -> psi[f(g)], g -> psi[Tgf(g)], g-> Tg psi[f(g)]
0 [-6.29  0.3  -1.6 ] [-5.01  0.74  1.28] [-5.01  0.74  1.28]
1 [-5.   -0.32 -2.34] [-4.74  0.77 -2.99] [-4.74  0.77 -2.99]
2 [-3.76 -0.25 -2.65] [-6.29  0.3  -1.6 ] [-6.29  0.3  -1.6 ]
3 [-5.6  -0.53 -0.63] [-5.   -0.32 -2.34] [-5.   -0.32 -2.34]
4 [-5.01  0.74  1.28] [-3.76 -0.25 -2.65] [-3.76 -0.25 -2.65]
5 [-4.74  0.77 -2.99] [-5.6  -0.53 -0.63] [-5.6  -0.53 -0.63]
